In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 6.1MB/s 
     |████████████████████████████████| 1.1MB 38.1MB/s 
     |████████████████████████████████| 890kB 53.9MB/s 
     |████████████████████████████████| 2.9MB 49.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b9aec396d3bae3a2803b5d1d60bc9c5024b9e93fc19666558ba7fbd97288e376
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random
import ast
from tqdm import tqdm
import re

import matplotlib.pyplot as plt
% matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel,AutoModelWithLMHead,AutoTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup


#GPU Connection

In [ ]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


#Reading & Preprocessing Dataset

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
data=pd.read_csv("/content/drive/MyDrive/Poem Generation/pg_data/pg_data_aldiwan/64k_dataset.csv")

In [ ]:
poems_data = data.loc[data['type_ar']=='عمودية']
poems_data=data=[ast.literal_eval(text) for text in tqdm(poems_data['poem_text'])]

100%|██████████| 64321/64321 [00:06<00:00, 10090.48it/s]


In [ ]:
all_data=[]
ARABIC_CHARS =   set(['ئ', 'ا', 'ب', 'ت', 'ة', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز',
                  'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م',
                  'ن', 'ه', 'و', 'ي', 'ء', 'آ', 'أ', 'ؤ', 'إ', 'ﺁ', 'ﺂ', 'ﺃ', 'ﺄ',
                  'ﺅ', 'ﺆ', 'ﺇ', 'ﺈ', 'ﺉ', 'ﺊ', 'ﺋ', 'ﺌ', 'ﺍ', 'ﺎ', 'ﺏ', 'ﺐ', 'ﺑ',
                  'ﺒ', 'ﺓ', 'ﺔ', 'ﺕ', 'ﺖ', 'ﺗ', 'ﺘ', 'ﺙ', 'ﺚ', 'ﺛ', 'ﺜ', 'ﺝ', 'ﺞ',
                  'ﺟ', 'ﺠ', 'ﺡ', 'ﺢ', 'ﺣ', 'ﺤ', 'ﺥ', 'ﺦ', 'ﺧ', 'ﺨ', 'ﺩ', 'ﺪ', 'ﺫ',
                  'ﺬ', 'ﺭ', 'ﺮ', 'ﺯ', 'ﺰ', 'ﺱ', 'ﺲ', 'ﺳ', 'ﺴ', 'ﺵ', 'ﺶ', 'ﺷ', 'ﺸ',
                  'ﺹ', 'ﺺ', 'ﺻ', 'ﺼ', 'ﺽ', 'ﺾ', 'ﺿ', 'ﻀ', 'ﻁ', 'ﻂ', 'ﻃ', 'ﻄ', 'ﻅ',
                  'ﻆ', 'ﻇ', 'ﻈ', 'ﻉ', 'ﻊ', 'ﻋ', 'ﻌ', 'ﻍ', 'ﻎ', 'ﻏ', 'ﻐ', 'ﻑ', 'ﻒ',
                  'ﻓ', 'ﻔ', 'ﻕ', 'ﻖ', 'ﻗ', 'ﻘ', 'ﻙ', 'ﻚ', 'ﻛ', 'ﻜ', 'ﻝ', 'ﻞ', 'ﻟ',
                  'ﻠ', 'ﻡ', 'ﻢ', 'ﻣ', 'ﻤ', 'ﻥ', 'ﻦ', 'ﻧ', 'ﻨ', 'ﻩ', 'ﻪ', 'ﻫ', 'ﻬ',
                  'ﻭ', 'ﻮ', 'ﻯ', 'ﻰ', 'ى', 'ﯼ', 'ﻱ', 'ﻲ', 'ﻳ', 'ﻴ', 'ﻵ', 'ﻶ', 'ﻷ',
                  'ﻸ', 'ﻹ', 'ﻺ', 'ﻻ', 'ﻼ'])


CHARS_PREVENT_REMOVING = set(ARABIC_CHARS)

STR_CHARS_PREVENT_REMOVING = ' '.join((c for c in CHARS_PREVENT_REMOVING))
ESCAPED_CHARS_PREVENT_REMOVING = re.escape(STR_CHARS_PREVENT_REMOVING)

# these only used in remove_non_arabic() method
def remove_non_arabic(text):
    return re.sub("[^%s]" % (ESCAPED_CHARS_PREVENT_REMOVING), '', text).strip()
"""
for i in range(len(poems_data)):
  if len(poems_data[i][0])==2:
    all_data.append(remove_non_arabic(poems_data[i][0][0]))
    all_data.append(remove_non_arabic(poems_data[i][0][1]))
  else:
    all_data.append(remove_non_arabic(poems_data[i][0][0]))

"""  

"""
for i in range(len(poems_data)):
  line1 = remove_non_arabic(poems_data[i][0][0])
  if len(poems_data[i][0]) == 2:
    line2 = remove_non_arabic(poems_data[i][0][1])
    all_data.extend((line1, line2))
  else:
    all_data.append(line1)

"""
for i in range(len(poems_data)):
  line1 = remove_non_arabic(poems_data[i][0][0])
  if len(poems_data[i][0]) == 2:
    line2 = remove_non_arabic(poems_data[i][0][1])
    #all_data.extend((line1, line2))
    all_data.append((line1 + ' - ' + line2))
  else:
    all_data.append(line1)


In [ ]:
sorted_data = sorted(all_data, key=len, reverse=True)

In [ ]:
sorted_data[0]

'متى يا حلوة النظرات والبسمات والإيماء والخطر - متى أملي على الآلام والحدثان والدهر'

In [ ]:
len(sorted_data)

64321

In [ ]:
sorted_data[0:5]

['متى يا حلوة النظرات والبسمات والإيماء والخطر - متى أملي على الآلام والحدثان والدهر',
 'عش ابق اسم سد قد جد مر انه رف اسر نل - غظ ارم صب احم اغز اسب رع زع دل اثن نل',
 'يا مي قومي فاندبن بسحرة شجو النوائح - كالحاملات الوقر بالثقل الملحت الدوالح',
 'أتوسوسين لنا بسر قد خبأت مدى الشتاء - هذى البراعم إذ تفتح لا يخالطها الرياء',
 'نحن في وادي وغيب الغيب في وادي عميق - والذي يدخل وادي الغيب منه لا يفيق']

#Tokenization

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('akhooli/gpt2-small-arabic', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
#tokenizer = AutoTokenizer.from_pretrained("akhooli/gpt2-small-arabic")

In [ ]:
print(' Original: ', sorted_data[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sorted_data[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sorted_data[0])))

 Original:  متى يا حلوة النظرات والبسمات والإيماء والخطر - متى أملي على الآلام والحدثان والدهر
Tokenized:  ['Ùħ', 'ØªÙī', 'ĠÙĬØ§', 'ĠØŃÙĦÙĪØ©', 'ĠØ§ÙĦÙĨØ¸Ø±', 'Ø§Øª', 'ĠÙĪØ§ÙĦØ¨', 'Ø³ÙħØ§Øª', 'ĠÙĪØ§ÙĦØ¥', 'ÙĬÙħ', 'Ø§Ø¡', 'ĠÙĪØ§ÙĦØ®', 'Ø·Ø±', 'Ġ', '-', 'ĠÙħØªÙī', 'ĠØ£Ùħ', 'ÙĦÙĬ', 'ĠØ¹ÙĦÙī', 'ĠØ§ÙĦØ¢ÙĦØ§Ùħ', 'ĠÙĪØ§ÙĦ', 'ØŃØ¯Ø«', 'Ø§ÙĨ', 'ĠÙĪØ§ÙĦØ¯', 'ÙĩØ±']
Token IDs:  [265, 565, 3596, 47687, 2963, 296, 1660, 25959, 1315, 419, 336, 2440, 675, 221, 13, 10938, 642, 334, 333, 27483, 318, 1653, 288, 1075, 495]


In [ ]:
max_len = 0

# For every sentence...
for sent in sorted_data:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  30


#Convert Dataset to Required Formatting

In [ ]:
batch_size = 2

In [ ]:
class GPT2Dataset(Dataset):

  def __init__(self, txt_list, tokenizer, gpt2_type="akhooli/gpt2-small-arabic", max_length=30):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:

      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx] 

In [ ]:
dataset = GPT2Dataset(sorted_data, tokenizer,30)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

57,888 training samples
6,433 validation samples


In [ ]:
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

#Load Model

In [ ]:
configuration = GPT2Config.from_pretrained('akhooli/gpt2-small-arabic', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("akhooli/gpt2-small-arabic", config=configuration)

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

In [ ]:
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

In [ ]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

#Train & Evaluate

In [ ]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                b_input_ids,
                do_sample=True, 
                max_length=120, 
                top_k=50, 
                top_p=0.95, 
                num_return_sequences=3,
                early_stopping=True,
                min_length=96,
                repetition_penalty=1.25
            )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids, 
#                            token_type_ids=None, 
                             attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   100  of  28,944. Loss: 3.580087900161743.   Elapsed: 0:00:06.
0: <|startoftext|>يا إخوتي شرح حالي بعدكم عجب - نأيتم فشهورياتك على ما بين العوام - منها -toftext|>أيقه في أن لا تنافيها من خائفا - مثل رسالتي أنا - عن الهوى طالعني في قلبي إلا لو مثلها في كل ضرورات أذرقت منك بالوفاء واعتراضا لي أسلامته
1: <|startoftext|>يا إخوتي شرح حالي بعدكم عجب - نأيتم فشهوريون في رسمه كتاب أدبtoftoftext|>ثقل الناس إلى ما هو أحسن منه - أم أنكرهها لو أذرف منه في مثله من أذكى ولا إذا أعجبت لي، فإن ليس على إلا ما أدنه مني فهو لا يفهم إلا في فضله الذي يلحبه بل أنه له إكرارا لا يرم حتى بسواها
2: <|startoftext|>يا إخوتي شرح حالي بعدكم عجب - نأيتم فشهوريها كل ما حالك من أروى - فما بدا فيك إلا أن أشقرني ما كان عليه في بعض الناس. قال عنه مسلم : من مثلها من عظمت منه إلا أنه لا يكاد يضر بها إلى العوامtoftextoftext|>أيا إذا نهمنا أتبعت لها على عقرب - وج
3: <|startoftext|>كانت قناتي لا تلين لغامز - فألانها الإصباح في ذهني فاقعهtotextft|>أنق كل منها نثارا يجاري ما هو من الهوىoft|>أيت على الناس منك طالبا ومن


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   200  of  28,944. Loss: 3.3201828002929688.   Elapsed: 0:00:15.
0: <|startoftext|>غنى الهزار على روض العرار بكم - فما عرفنا من كل جانب في إكتاق ما فيه
1: <|startoftext|>غنى الهزار على روض العرار بكم - فما عرفنا من إفاق العاذى به حتى قد لم يزل عاذا
2: <|startoftext|>غنى الهزار على روض العرار بكم - فما عرفنا من ن وما أزل إلا ذاك النحر فيه
3: <|startoftext|>رضيت برضوى روضة ومناخا - فإن به مرعى وفيه عذار
4: <|startoftext|>رضيت برضوى روضة ومناخا - فإن به مرعى وفيه
5: <|startoftext|>رضيت برضوى روضة ومناخا - فإن به مرعى وفيه ظام


...

  Batch 28,700  of  28,944. Loss: 1.1012712717056274.   Elapsed: 0:40:14.
0: <|startoftext|>قد هاج حزني وعادني ذكري - يوم التقينا عشية النفر غداة النفر خرجن برمت جانباه
1: <|startoftext|>قد هاج حزني وعادني ذكري - يوم التقينا عشية النفر فأعداني الشوق نفسي
2: <|startoftext|>قد هاج حزني وعادني ذكري - يوم التقينا عشية النفر خرززا وأنى متنصداغه
3: <|startoftext|>يا ضارب الجيش بي في وسط مفرقه - لقد ضربت بعينيه كفي - على القلب المعنىكت أدبه وتلافيني به الروح والنجم واهجر هجرا
4: <|startoftext|>يا ضارب الجيش بي في وسط مفرقه - لقد ضربت بعينيك من عرصةيشي بلا ظلال
5: <|startoftext|>يا ضارب الجيش بي في وسط مفرقه - لقد ضربت بعينيه جنان من الدماء محتدنا قلم المراسيونا


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 28,800  of  28,944. Loss: 1.513747215270996.   Elapsed: 0:40:22.
0: <|startoftext|>والطل ينشر كل وقت - كيومنا لؤلؤا فيها سقي
1: <|startoftext|>والطل ينشر كل وقت - كيومنا لؤلؤا فيها سقي
2: <|startoftext|>والطل ينشر كل وقت - كيومنا لؤلؤا فيها سقي
3: <|startoftext|>أيها المنصف إلا رجلا - واحدا أصبحت ممن ظلمه
4: <|startoftext|>أيها المنصف إلا رجلا - واحدا أصبحت ممن ظلمه
5: <|startoftext|>أيها المنصف إلا رجلا - واحدا أصبحت ممن ظلمه


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 28,900  of  28,944. Loss: 1.6611708402633667.   Elapsed: 0:40:30.
0: <|startoftext|>في مهمه قطع السرى أنياطها - قطعا وزادهم العيان على النظام والبخل المصائب عاذل هتمل المخازي علي
1: <|startoftext|>في مهمه قطع السرى أنياطها - قطعا وزادهم العيان كأنا على البأس كدر تمادى بك عان أوريت أيادي لم يعدو الأقلام
2: <|startoftext|>في مهمه قطع السرى أنياطها - قطعا وزادهم العيان وأنهدها أني تضاهي البكايدك ربها أني تضن مرادي
3: <|startoftext|>لك يا أمير المؤمنين يد - في الظالمين وأخذهم لبقه
4: <|startoftext|>لك يا أمير المؤمنين يد - في الظالمين وأخذهم لبقه
5: <|startoftext|>لك يا أمير المؤمنين يد - في الظالمين وأخذهم لبقه

  Average training loss: 1.56
  Training epoch took: 0:40:36

Running Validation...
  Validation Loss: 3.24
  Validation took: 0:00:41

Training complete!
Total training took 3:24:54 (h:mm:ss)


#Save Model

In [ ]:
output_dir = '/content/drive/MyDrive/model_save64k_gpt2'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to /content/drive/MyDrive/model_save64k_gpt2


('/content/drive/MyDrive/model_save64k_gpt2/tokenizer_config.json',
 '/content/drive/MyDrive/model_save64k_gpt2/special_tokens_map.json',
 '/content/drive/MyDrive/model_save64k_gpt2/vocab.json',
 '/content/drive/MyDrive/model_save64k_gpt2/merges.txt',
 '/content/drive/MyDrive/model_save64k_gpt2/added_tokens.json')